In [1]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import scipy.io
import pandas as pd
import csv
from scipy.fftpack import dct, idct
from scipy.optimize import minimize

In [2]:
def cosamp(phi, u, s, epsilon=1e-10, max_iter=1000):
    """
    Return an `s`-sparse approximation of the target signal
    Input:
        - phi, sampling matrix
        - u, noisy sample vector
        - s, sparsity
    """
    a = np.zeros(phi.shape[1])
    v = u
    it = 0 # count
    halt = False
    while not halt:
        it += 1
        print("Iteration {}\r".format(it), end="")
        
        y = np.dot(np.transpose(phi), v)
        omega = np.argsort(y)[-(2*s):] # large components
        omega = np.union1d(omega, a.nonzero()[0]) # use set instead?
        phiT = phi[:, omega]
        b = np.zeros(phi.shape[1])
        # Solve Least Square
        b[omega], _, _, _ = np.linalg.lstsq(phiT, u)
        
        # Get new estimate
        b[np.argsort(b)[:-s]] = 0
        a = b
        # Halt criterion
        v_old = v
        v = u - np.dot(phi, a)

        halt = (np.linalg.norm(v - v_old) < epsilon) or \
            np.linalg.norm(v) < epsilon or \
            it > max_iter
        
    return a

In [ ]:
n = 16000 #sampling rate, just an example

df = pd.read_csv('acousticsampledata.csv')    
acoustic = df.to_numpy()
acoustic = np.reshape(acoustic, 628511)
acou = acoustic[0:16000]
df = pd.read_csv('seismicsampledata.csv')    
seismic = df.to_numpy()
seismic = np.reshape(seismic, 628511)

acoufft = np.fft.fft(acoustic[:16000])
seisfft = np.fft.fft(seismic[:16000])

PSD = acoufft * np.conj(acoufft) / 16000 # Power spectral density

Psi = dct(np.identity(n))

p = 4096  # num. random samples, p = n/32
perm = np.floor(np.random.rand(p) *n).astype(int) #random gaussian matrix
y = acou[perm]

Theta = Psi[perm,:]

s = cosamp(Theta,y,4096,epsilon=1.e-10,max_iter=10) # CS via matching pursuit
xrecon = idct(s) # reconstruct full signal

print(xrecon.shape)
mean = np.mean(acou)

s = 0
for i in range(16000):
    a = np.abs(acou[i] - xrecon[i])
    s = s + a*a
s = s/n
s = np.sqrt(s)

plt.plot(xrecon)
plt.show()
plt.plot(acou[:16000])
plt.show()
print(s, mean, s/mean)